# New data pipeline

Input: Uniprot, GO graph, UniprotGOA, root node
Output: A list of protein sequences and go annotations, including ancestors, under the root node

Next: Innner merge that dataset with GO to ChEBI annotations
Then: Create Matrices for ChEBI terms and GO terms

In [1]:
from subpred.util import load_df
import networkx as nx
import pandas as pd
from subpred.transmembrane_transporters import get_transmembrane_transporter_dataset

# ORGANISM_IDS = {559292}
ORGANISM_IDS = None
SWISSPROT_ONLY = False
MAX_SEQUENCE_EVIDENCE_CODE = 2
EXCLUDE_IEA_GO_TERMS = False
REMOVE_PROTEINS_WITHOUT_GENE_NAMES = False

df_sequences, df_uniprot_goa, df_go_chebi = get_transmembrane_transporter_dataset(
    organism_ids=ORGANISM_IDS,
    swissprot_only=SWISSPROT_ONLY,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=EXCLUDE_IEA_GO_TERMS,
    max_sequence_evidence_code=MAX_SEQUENCE_EVIDENCE_CODE,
    remove_proteins_without_gene_names=REMOVE_PROTEINS_WITHOUT_GENE_NAMES,
)

In [2]:
df_uniprot_goa

,Uniprot,qualifier,go_id,go_term,evidence_code,aspect,go_id_ancestor,go_term_ancestor
0,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0022857,transmembrane transporter activity
1,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0015267,channel activity
2,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0022803,passive transmembrane transporter activity
3,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0015288,porin activity
4,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0022829,wide pore channel activity
...,...,...,...,...,...,...,...,...
1005409,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0022853,active monoatomic ion transmembrane transporte...
1005410,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0022804,active transmembrane transporter activity
1005411,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0022890,inorganic cation transmembrane transporter act...
1005412,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0015370,solute:sodium symporter activity


In [3]:
go_chebi = load_df("go_chebi")

go_chebi = go_chebi[go_chebi.go_id.isin(df_uniprot_goa.go_id_ancestor)]
go_chebi = go_chebi[go_chebi.relation == "has_primary_input"]
go_chebi

,go_id,chebi_id,chebi_term,relation
50,GO:0015078,CHEBI:24636,proton,has_primary_input
54,GO:0042958,CHEBI:25140,maltodextrin,has_primary_input
175,GO:0015075,CHEBI:24867,monoatomic ion,has_primary_input
177,GO:0015085,CHEBI:29108,calcium(2+),has_primary_input
321,GO:0005249,CHEBI:29103,potassium(1+),has_primary_input
...,...,...,...,...
21537,GO:0009674,CHEBI:29103,potassium(1+),has_primary_input
21543,GO:0010542,CHEBI:17632,nitrate,has_primary_input
21624,GO:1905131,CHEBI:192797,carcininium,has_primary_input
21698,GO:0015445,CHEBI:60253,silver cation,has_primary_input


In [4]:
graph_chebi = load_df("chebi_obo")
graph_chebi = graph_chebi.edge_subgraph(
    edges=[
        (source, sink, key)
        for source, sink, key in graph_chebi.edges(keys=True)
        if key == "is_a"
    ]
)

In [5]:
go_chebi["chebi_id_ancestor"] = go_chebi.chebi_id.transform(
    lambda x: set(nx.descendants(graph_chebi, x) | {x})
)
go_chebi = go_chebi.explode("chebi_id_ancestor")
chebi_id_to_term = {k: v for k, v in graph_chebi.nodes(data="name")}
go_chebi["chebi_term_ancestor"] = go_chebi.chebi_id_ancestor.map(chebi_id_to_term)
go_chebi = go_chebi.reset_index(drop=True)

In [6]:
chebi_id_to_properties = {
    chebi_id: properties_list
    for chebi_id, properties_list in graph_chebi.nodes(data="property_value")
    if properties_list
}

In [7]:
go_chebi

,go_id,chebi_id,chebi_term,relation,chebi_id_ancestor,chebi_term_ancestor
0,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:36344,hadron
1,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:24431,chemical entity
2,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:36916,cation
3,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:33253,nucleon
4,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:24867,monoatomic ion
...,...,...,...,...,...,...
14756,GO:0042943,CHEBI:59871,D-alpha-amino acid zwitterion,has_primary_input,CHEBI:36357,polyatomic entity
14757,GO:0042943,CHEBI:59871,D-alpha-amino acid zwitterion,has_primary_input,CHEBI:33579,main group molecular entity
14758,GO:0042943,CHEBI:59871,D-alpha-amino acid zwitterion,has_primary_input,CHEBI:33582,carbon group molecular entity
14759,GO:0042943,CHEBI:59871,D-alpha-amino acid zwitterion,has_primary_input,CHEBI:27369,zwitterion


In [8]:
go_chebi = go_chebi[
    go_chebi.chebi_id_ancestor.isin(chebi_id_to_properties.keys())
].reset_index(drop=True)

go_chebi

,go_id,chebi_id,chebi_term,relation,chebi_id_ancestor,chebi_term_ancestor
0,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:15378,hydron
1,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:24636,proton
2,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:33251,monoatomic hydrogen
3,GO:0015078,CHEBI:24636,proton,has_primary_input,CHEBI:25414,monoatomic monocation
4,GO:0042958,CHEBI:25140,maltodextrin,has_primary_input,CHEBI:37163,glucan
...,...,...,...,...,...,...
1290,GO:0009674,CHEBI:29103,potassium(1+),has_primary_input,CHEBI:25414,monoatomic monocation
1291,GO:0010542,CHEBI:17632,nitrate,has_primary_input,CHEBI:17632,nitrate
1292,GO:1905131,CHEBI:192797,carcininium,has_primary_input,CHEBI:65296,primary ammonium ion
1293,GO:1905131,CHEBI:192797,carcininium,has_primary_input,CHEBI:192797,carcininium


In [9]:
graph_go = load_df("go_obo")
go_term_to_name = {k: v for k, v in graph_go.nodes(data="name")}

In [10]:
go_chebi.insert(column="go_term", value=go_chebi.go_id.map(go_term_to_name), loc=1)

In [11]:
go_chebi.drop_duplicates()

,go_id,go_term,chebi_id,chebi_term,relation,chebi_id_ancestor,chebi_term_ancestor
0,GO:0015078,proton transmembrane transporter activity,CHEBI:24636,proton,has_primary_input,CHEBI:15378,hydron
1,GO:0015078,proton transmembrane transporter activity,CHEBI:24636,proton,has_primary_input,CHEBI:24636,proton
2,GO:0015078,proton transmembrane transporter activity,CHEBI:24636,proton,has_primary_input,CHEBI:33251,monoatomic hydrogen
3,GO:0015078,proton transmembrane transporter activity,CHEBI:24636,proton,has_primary_input,CHEBI:25414,monoatomic monocation
4,GO:0042958,maltodextrin transmembrane transporter activity,CHEBI:25140,maltodextrin,has_primary_input,CHEBI:37163,glucan
...,...,...,...,...,...,...,...
1290,GO:0009674,potassium:sodium symporter activity,CHEBI:29103,potassium(1+),has_primary_input,CHEBI:25414,monoatomic monocation
1291,GO:0010542,nitrate efflux transmembrane transporter activity,CHEBI:17632,nitrate,has_primary_input,CHEBI:17632,nitrate
1292,GO:1905131,carcinine transmembrane transporter activity,CHEBI:192797,carcininium,has_primary_input,CHEBI:65296,primary ammonium ion
1293,GO:1905131,carcinine transmembrane transporter activity,CHEBI:192797,carcininium,has_primary_input,CHEBI:192797,carcininium


In [12]:
# TODO filter chebi ancestors for smiles or formula or mass presence

In [13]:
go_chebi_direct = (
    go_chebi[["go_term", "chebi_term_ancestor"]]
    .groupby("go_term")
    .apply(lambda x: [term for term in x.chebi_term_ancestor.tolist() if term == term])
    .to_frame(name="substrates")
    .reset_index()
)

go_chebi_direct

,go_term,substrates
0,(+)-abscisic acid D-glucopyranosyl ester trans...,"[carboxylic ester, (+)-abscisic acid D-glucopy..."
1,(R)-carnitine transmembrane transporter activity,"[(R)-carnitine, carnitine]"
2,(R)-carnitine:4-(trimethylammonio)butanoate an...,"[(R)-carnitine, carnitine, 4-(trimethylammonio..."
3,2-keto-3-deoxygluconate:proton symporter activity,"[monocarboxylic acid anion, 2-oxo monocarboxyl..."
4,3'-phosphoadenosine 5'-phosphosulfate transmem...,"[organosulfate oxoanion, 3'-phosphonato-5'-ade..."
...,...,...
429,xanthine transmembrane transporter activity,"[purine nucleobase, purines, xanthine]"
430,xanthosine transmembrane transporter activity,"[nucleoside, purines, purines D-ribonucleoside..."
431,zinc ion transmembrane transporter activity,"[zinc(2+), monoatomic dication]"
432,zinc:bicarbonate symporter activity,[hydrogencarbonate]


In [14]:
uniprot_go_chebi = df_uniprot_goa.merge(
    go_chebi_direct, how="left", left_on="go_term_ancestor", right_on="go_term"
)

uniprot_go_chebi

,Uniprot,qualifier,go_id,go_term_x,evidence_code,aspect,go_id_ancestor,go_term_ancestor,go_term_y,substrates
0,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0022857,transmembrane transporter activity,NaN,NaN
1,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0015267,channel activity,NaN,NaN
2,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0022803,passive transmembrane transporter activity,NaN,NaN
3,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0015288,porin activity,NaN,NaN
4,A0A014M993,enables,GO:0015288,porin activity,IEA,F,GO:0022829,wide pore channel activity,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1005409,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0022853,active monoatomic ion transmembrane transporte...,NaN,NaN
1005410,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0022804,active transmembrane transporter activity,NaN,NaN
1005411,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0022890,inorganic cation transmembrane transporter act...,NaN,NaN
1005412,Z4YKJ7,enables,GO:0015501,glutamate:sodium symporter activity,IEA,F,GO:0015370,solute:sodium symporter activity,solute:sodium symporter activity,"[sodium(1+), monoatomic monocation]"


In [15]:
go_chebi_direct = uniprot_go_chebi[["go_term_ancestor", "substrates"]]

go_chebi_direct = go_chebi_direct.assign(
    substrates=go_chebi_direct.substrates.apply(lambda x: frozenset(x) if x == x else x)
)
go_chebi_direct = go_chebi_direct.drop_duplicates().reset_index(drop=True)
go_chebi_direct

,go_term_ancestor,substrates
0,transmembrane transporter activity,NaN
1,channel activity,NaN
2,passive transmembrane transporter activity,NaN
3,porin activity,NaN
4,wide pore channel activity,NaN
...,...,...
801,high-affinity copper ion transmembrane transpo...,NaN
802,thiamine:proton symporter activity,NaN
803,high-affinity thiamine:proton symporter activity,NaN
804,carcinine transmembrane transporter activity,"(primary ammonium ion, carcininium)"


In [16]:
print(
    "chebi terms available for",
    go_chebi_direct.substrates.value_counts(dropna=True).sum(),
)
# go_chebi_direct[go_chebi_direct.substrates.isna()].to_csv("test.tsv", sep="\t")
# non_annotated = go_chebi_direct[go_chebi_direct.substrates.isna()]

chebi terms available for 434


In [33]:
go_chebi_direct[
    go_chebi_direct.go_term_ancestor.str.contains("monoatomic ion")
    & go_chebi_direct.substrates.isna()
]  # .substrates = "asdf"

,go_term_ancestor,substrates


In [ ]:
# assign by keywords
# TODO 

In [36]:
import numpy as np

searchterm_to_chebi = {
    "protein": "CHEBI:36080",
    "ion": "CHEBI:24870",
    "anion": "CHEBI:22563",
    "cation": "CHEBI:36916",
    # "monoatomic ion": "CHEBI:24867",
    "monoatomic anion": "CHEBI:23905",
    "monoatomic cation": "CHEBI:23906",
    "proton" : "CHEBI:24636" 
}


for key, val in searchterm_to_chebi.items():
    go_chebi_direct = go_chebi_direct.assign(
        substrates=np.where(
            go_chebi_direct.go_term_ancestor.str.contains(key)
            & go_chebi_direct.substrates.isna(),
            val,
            go_chebi_direct.substrates,
        )
    )

In [37]:
go_chebi_direct

,go_term_ancestor,substrates
0,transmembrane transporter activity,NaN
1,channel activity,NaN
2,passive transmembrane transporter activity,NaN
3,porin activity,NaN
4,wide pore channel activity,NaN
...,...,...
801,high-affinity copper ion transmembrane transpo...,CHEBI:24870
802,thiamine:proton symporter activity,CHEBI:24636
803,high-affinity thiamine:proton symporter activity,CHEBI:24636
804,carcinine transmembrane transporter activity,"(primary ammonium ion, carcininium)"


# OLD CODE

In [1]:
from subpred.transmembrane_transporters import get_transmembrane_transporter_dataset

df_sequences, df_uniprot_goa, df_go_chebi = get_transmembrane_transporter_dataset(
    organism_ids={9606},
    swissprot_only=False,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=False,
    max_sequence_evidence_code=1,
)
df_uniprot_goa

,Uniprot,qualifier,go_id,go_term,evidence_code,aspect,go_id_ancestor,go_term_ancestor
0,A0A024RCG2,enables,GO:0022857,transmembrane transporter activity,IEA,F,GO:0022857,transmembrane transporter activity
1,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0140359,ABC-type transporter activity
2,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0022857,transmembrane transporter activity
3,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0015399,primary active transmembrane transporter activity
4,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0042626,ATPase-coupled transmembrane transporter activity
...,...,...,...,...,...,...,...,...
67172,X5CMH5,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0042626,ATPase-coupled transmembrane transporter activity
67173,X5CMH5,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0022804,active transmembrane transporter activity
67174,X5CMH5,enables,GO:1904680,peptide transmembrane transporter activity,IEA,F,GO:0042887,amide transmembrane transporter activity
67175,X5CMH5,enables,GO:1904680,peptide transmembrane transporter activity,IEA,F,GO:1904680,peptide transmembrane transporter activity


In [5]:
df_go_chebi

,go_id,chebi_id,chebi_term,chebi_go_relation
0,GO:0000064,CHEBI:46911,L-ornithinium(1+),has_primary_input
1,GO:0000095,CHEBI:59789,S-adenosyl-L-methionine zwitterion,has_primary_input
2,GO:0000099,CHEBI:26834,sulfur-containing amino acid,has_primary_input
3,GO:0000295,CHEBI:61293,adenyl nucleotide,has_primary_input
4,GO:0000514,CHEBI:14321,glutamate(1-),has_primary_input
...,...,...,...,...
693,GO:1901480,CHEBI:30823,oleate,has_primary_input
694,GO:1901505,CHEBI:63299,carbohydrate derivative,has_primary_input
695,GO:1901682,CHEBI:26835,sulfur molecular entity,has_primary_input
696,GO:1902282,CHEBI:29103,potassium(1+),has_primary_input


In [ ]:
df_go_chebi_primary = df_go_chebi[
    df_go_chebi.chebi_go_relation == "has_primary_input"
].reset_index(drop=True)
df_go_chebi_primary = df_go_chebi_primary[
    df_go_chebi_primary.chebi_id.isin(chebi_id_to_properties.keys())
]
df_go_chebi_primary

## Merge

In [3]:
import pandas as pd

df_uniprot_go_transporter = pd.merge(
    df_uniprot_goa, df_sequences, left_on="Uniprot", right_index=True, how="inner"
).reset_index(drop=True)
df_uniprot_go_transporter

,Uniprot,qualifier,go_id,go_term,evidence_code,aspect,go_id_ancestor,go_term_ancestor,sequence,reviewed,protein_existence,organism_id,protein_names
0,A0A024RCG2,enables,GO:0022857,transmembrane transporter activity,IEA,F,GO:0022857,transmembrane transporter activity,MQGARAPRDQGQSPGRMSALGRSSVILLTYVLAATELTCLFMQFSI...,False,1,9606,Solute carrier family 22 (Organic cation trans...
1,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0140359,ABC-type transporter activity,MSTAIREVGVWRQTRTLLLKNYLIKCRTKKSSVQEILFPLFFLFWL...,False,1,9606,Cholesterol transporter ABCA5
2,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0022857,transmembrane transporter activity,MSTAIREVGVWRQTRTLLLKNYLIKCRTKKSSVQEILFPLFFLFWL...,False,1,9606,Cholesterol transporter ABCA5
3,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0015399,primary active transmembrane transporter activity,MSTAIREVGVWRQTRTLLLKNYLIKCRTKKSSVQEILFPLFFLFWL...,False,1,9606,Cholesterol transporter ABCA5
4,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0042626,ATPase-coupled transmembrane transporter activity,MSTAIREVGVWRQTRTLLLKNYLIKCRTKKSSVQEILFPLFFLFWL...,False,1,9606,Cholesterol transporter ABCA5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67172,X5CMH5,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0042626,ATPase-coupled transmembrane transporter activity,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,False,1,9606,Antigen peptide transporter 2 (TAP2)
67173,X5CMH5,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0022804,active transmembrane transporter activity,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,False,1,9606,Antigen peptide transporter 2 (TAP2)
67174,X5CMH5,enables,GO:1904680,peptide transmembrane transporter activity,IEA,F,GO:0042887,amide transmembrane transporter activity,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,False,1,9606,Antigen peptide transporter 2 (TAP2)
67175,X5CMH5,enables,GO:1904680,peptide transmembrane transporter activity,IEA,F,GO:1904680,peptide transmembrane transporter activity,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,False,1,9606,Antigen peptide transporter 2 (TAP2)


In [4]:
df_uniprot_goa_chebi = df_uniprot_goa.merge(
    df_go_chebi, how="left", left_on="go_id_ancestor", right_on="go_id"
)
df_uniprot_goa_chebi

,Uniprot,qualifier,go_id_x,go_term,evidence_code,aspect,go_id_ancestor,go_term_ancestor,go_id_y,chebi_id,chebi_term,chebi_go_relation
0,A0A024RCG2,enables,GO:0022857,transmembrane transporter activity,IEA,F,GO:0022857,transmembrane transporter activity,NaN,NaN,NaN,NaN
1,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0140359,ABC-type transporter activity,NaN,NaN,NaN,NaN
2,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0022857,transmembrane transporter activity,NaN,NaN,NaN,NaN
3,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0015399,primary active transmembrane transporter activity,NaN,NaN,NaN,NaN
4,A0A075B778,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0042626,ATPase-coupled transmembrane transporter activity,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
75020,X5CMH5,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0042626,ATPase-coupled transmembrane transporter activity,NaN,NaN,NaN,NaN
75021,X5CMH5,enables,GO:0140359,ABC-type transporter activity,IEA,F,GO:0022804,active transmembrane transporter activity,NaN,NaN,NaN,NaN
75022,X5CMH5,enables,GO:1904680,peptide transmembrane transporter activity,IEA,F,GO:0042887,amide transmembrane transporter activity,GO:0042887,CHEBI:32988,amide,has_primary_input
75023,X5CMH5,enables,GO:1904680,peptide transmembrane transporter activity,IEA,F,GO:1904680,peptide transmembrane transporter activity,GO:1904680,CHEBI:16670,peptide,has_primary_input
